In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("../data/GMS_ProjectDataset_V2.csv")
df.head()

,person_id,GenderCategory,RaceCategory,EthnicityCategory,Age,Region,Glipizide_Final,Glimepiride_Final,Glyburide_Final,Metformin_Final,...,Alzheimer_Final,FootUlcer_Final,Abnormal_Glucose_Final,DMScreen_Final,A1C_Final,GlucoseTest_Final,InsulinTest_Final,DM_Drugs,AnyLab,Diabetes_Indicator
0,1,1,1,1,100,2,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0.0
1,2,1,1,1,80,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,1,1.0
2,3,2,1,1,87,1,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1.0
3,4,1,3,2,82,4,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1.0
4,5,1,1,1,87,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1.0


In [3]:
columns_with_nan = df.columns[df.isin(['', np.nan]).any()]
columns_with_nan

Index(['Glipizide_Final', 'Glimepiride_Final', 'Glyburide_Final',
       'Metformin_Final', 'Pioglitazone_Final', 'Rosiglitazone_Final',
       'Beta_Blockers_Final', 'ACE_Inhibitors_Final', 'ARB_Final',
       'Diuretics_Final', 'PPI_Final', 'Levothyroxine_Final', 'CCB_Final',
       'Vasodilators_Final', 'Alpha_Blockers_Final',
       'Centrally_Acting_Agents_Final', 'Statins_Final',
       'Anti_Platelets_Final', 'Anti_Coagulants_Final', 'Steroids_Final',
       'Heart_Disease_Final', 'Hypothyroid_Final', 'Anemia_Final',
       'Kidney_Disease_Final', 'GERD_Final', 'Neuropathy_Final',
       'Eye_Disorder_Final', 'Atherosclerosis_Final', 'Alzheimer_Final',
       'FootUlcer_Final', 'Abnormal_Glucose_Final', 'DMScreen_Final',
       'A1C_Final', 'GlucoseTest_Final', 'InsulinTest_Final',
       'Diabetes_Indicator'],
      dtype='object')

In [4]:
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

# Initialize the Linear Regression model
model = LinearRegression()

# Define the columns with missing values
columns_with_missing = columns_with_nan[:-1]

# Impute missing values using SimpleImputer
#imputer = SimpleImputer(strategy='KNN') 
imputer = KNNImputer(n_neighbors=5)
#for col in columns_with_missing:
for col in columns_with_missing:
    df[col] = imputer.fit_transform(df[[col]]).round().astype(int)


In [5]:
columns_with_nan = df.columns[df.isin(['', np.nan]).any()]
columns_with_nan

Index(['Diabetes_Indicator'], dtype='object')

In [6]:
from sklearn.linear_model import LogisticRegression

# Define the column with missing values
column_with_missing = 'Diabetes_Indicator'

# Create a copy of the DataFrame for imputation
df_copy = df.copy()

# Split the data into features (X) and target (y)
X = df_copy.drop(columns=[column_with_missing])  # Features
y = df_copy[column_with_missing]  # Target column

# Identify rows with missing values in the target column
missing_rows = y.isna()

# Initialize the Logistic Regression model
logistic_model = LogisticRegression(max_iter = 2000)

# Fit the logistic regression model on non-missing data
logistic_model.fit(X[~missing_rows], y[~missing_rows])

# Predict and impute missing values for the target column
imputed_values = logistic_model.predict(X[missing_rows])
df.loc[missing_rows, column_with_missing] = imputed_values

# Now, the missing values in the 'Diabetes_Indicator' column have been imputed using logistic regression


In [7]:
columns_with_nan = df.columns[df.isin(['', np.nan]).any()]
columns_with_nan

Index([], dtype='object')

In [8]:
print(df['Diabetes_Indicator'].unique())

[0. 1.]


In [9]:
from sklearn.model_selection import train_test_split

In [11]:
# Split the dataset into features (X) and target (y)
X = df.drop(columns=['Diabetes_Indicator'])
y = df['Diabetes_Indicator']

# Split the dataset into training (2/3) and testing (1/3) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

In [13]:
X_train.to_csv("../outputs/imputed_X_train.csv", index=False)
X_test.to_csv("../outputs/imputed_X_test.csv", index=False)
y_train.to_csv("../outputs/imputed_y_train.csv", index=False)
y_test.to_csv("../outputs/imputed_y_test.csv", index=False)